In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import torch
from torch import nn
import random

from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

from torch.utils.tensorboard import SummaryWriter

import os

from datetime import datetime

In [ ]:
data_path = '/content/drive/MyDrive/data/'

In [ ]:
train_betas_path = data_path + 'nsd_train_fmriavg_nsdgeneral_sub1.npy'
train_caps_embed_path = data_path + 'train_caps_embeds_sub1.npy'
train_caps_embed_neg_path = data_path + 'train_caps_embeds_negative_sub1.npy'

# train_betas = np.load(train_betas_path)
# train_caps_embed = np.load(train_caps_embed_path)
# train_caps_embed_neg = np.load(train_caps_embed_neg_path)

In [ ]:
test_betas_path = data_path + 'test_betas_split_sub1.npy'
test_caps_embed_path = data_path + 'test_caps_embeds_split_sub1.npy'
test_caps_embed_neg_path = data_path + 'test_caps_embeds_negative_split_sub1.npy'

# test_betas = np.load(test_betas_path)
# test_caps_embed = np.load(test_caps_embed_path)
# test_caps_embed_neg = np.load(test_caps_embed_neg_path)

In [ ]:
val_betas_path = data_path + 'val_betas_split_sub1.npy'
val_caps_embed_path = data_path + 'val_caps_embeds_split_sub1.npy'
val_caps_embed_neg_path = data_path + 'val_caps_embeds_split_negative_sub1.npy'

# test_betas = np.load(test_betas_path)
# test_caps_embed = np.load(test_caps_embed_path)
# test_caps_embed_neg = np.load(test_caps_embed_neg_path)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, betas_path, embeds_path):
        self.betas = torch.from_numpy(np.load(betas_path)).float().cuda()
        self.embeds = torch.from_numpy(
                np.squeeze(np.load(embeds_path))
            ).float().cuda()

    def __len__(self):
        return len(self.betas)

    def __getitem__(self, index):
        return self.betas[index], self.embeds[index]


class CustomDataLoader:
    def __init__(self, dataset, batch_size):
        self.data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    def __iter__(self):
        return iter(self.data_loader)

    def __len__(self):
        return len(self.data_loader)

batch_size = 64

train_dataset = CustomDataset(train_betas_path,
                              train_caps_embed_path,)
train_data_loader = CustomDataLoader(train_dataset, batch_size)

test_dataset = CustomDataset(test_betas_path,
                             test_caps_embed_path,)
test_data_loader = CustomDataLoader(test_dataset, batch_size)

val_dataset = CustomDataset(val_betas_path,
                            val_caps_embed_path,)
val_data_loader = CustomDataLoader(val_dataset, batch_size)

for beta, embed in train_data_loader:
    print(f"beta      {beta.shape}")
    print(f"embed     {embed.shape}")
    break

beta      torch.Size([64, 15724])
embed     torch.Size([64, 1280])


In [ ]:
input_dim = 15724
print('input_dim:', input_dim)
ouput_dim = 1280
print('output_dim:', ouput_dim)

input_dim: 15724
output_dim: 1280


In [ ]:
class RegressionModel(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.linear_regression = nn.Linear(in_features=input_channels,
                               out_features=output_channels)
        self.layer0 = 2**10 * 12
        self.layer1 = 2**10 * 10
        self.layer2 = 2**10 * 5
        self.layer3 = 2**10 * 2

        self.mlp = nn.Sequential(
            nn.Linear(in_features=input_channels,
                      out_features=self.layer0),
            nn.LeakyReLU(0.05, inplace=True),
            nn.Linear(in_features=self.layer0,
                      out_features=self.layer1),
            nn.LeakyReLU(0.05, inplace=True),
            nn.Linear(in_features=self.layer1,
                      out_features=self.layer2),
            nn.LeakyReLU(0.05, inplace=True),
            nn.Linear(in_features=self.layer2,
                      out_features=self.layer3),
            nn.LeakyReLU(0.05, inplace=True),
            nn.Linear(in_features=self.layer3,
                      out_features=output_channels),
        )


    def forward(self, x):
        return self.mlp(x)

model_embed = RegressionModel(input_dim, ouput_dim).cuda()

In [ ]:
import torch.nn.functional as F

class CosineSimilarityLoss(nn.Module):
    def __init__(self):
        super(CosineSimilarityLoss, self).__init__()

    def forward(self, y_true, y_pred):
        cosine_sim = F.cosine_similarity(y_true, y_pred, dim=1)
        loss = 1 - cosine_sim.mean()
        return loss

In [ ]:
cos_loss = CosineSimilarityLoss()
mse_loss = nn.MSELoss(reduction='mean')
optim_embed = torch.optim.Adam(
    params=model_embed.parameters(), lr=0.0001, weight_decay=0.01)

In [ ]:
run_name = 'mlp_one_model' #+ datetime.now().strftime("%Y-%m-%d_%H:%M")
writer = SummaryWriter(os.path.join(data_path, 'runs', run_name))

In [ ]:
import torch
from tqdm import tqdm

num_epochs = 100
iter_index = 0
min_test_loss = 1e15
best_model_path = os.path.join(data_path, f'../models/{run_name}.pth')
test_iter_index = 0

for epoch in tqdm(range(num_epochs)):
    model_embed.train()
    for betas, embeds in train_data_loader:
        iter_index += 1
        pred_embeds = model_embed(betas)

        loss_embed = mse_loss(pred_embeds, embeds)
        optim_embed.zero_grad()
        loss_embed.backward()
        optim_embed.step()

        writer.add_scalars('train/losses', {
            'embed': loss_embed.item(),
        }, iter_index)

    test_loss = 0
    model_embed.eval()
    with torch.inference_mode():
        n_batches = 0
        for betas, embeds in val_data_loader:
            test_iter_index += 1
            pred_embeds = model_embed(betas)

            loss_embed = mse_loss(pred_embeds, embeds)

            cos_embed = cos_loss(pred_embeds, embeds)

            writer.add_scalars('val/losses', {
                'embed': loss_embed.item(),
            }, test_iter_index)

            writer.add_scalars('val/cos_sim', {
                'cos_embed': cos_embed.item(),
            }, test_iter_index)

            n_batches += 1
            test_loss += loss_embed.item()

    test_loss /= n_batches
    print(test_loss)
    if test_loss < min_test_loss:
        min_test_loss = test_loss
        print(f'Model saved with loss: {test_loss}')
        torch.save({
            'model_embed_state_dict': model_embed.state_dict(),
            'epoch': epoch,
            'min_test_loss': min_test_loss,
        }, best_model_path)

  0%|          | 0/100 [00:00<?, ?it/s]

0.8785734251141548
Model saved with loss: 0.8785734251141548


  1%|          | 1/100 [00:30<50:34, 30.65s/it]

0.876572422683239
Model saved with loss: 0.876572422683239


  2%|▏         | 2/100 [01:05<54:11, 33.18s/it]

...

0.6745445132255554


 99%|█████████▉| 99/100 [50:59<00:23, 23.78s/it]

0.6929002478718758


100%|██████████| 100/100 [51:23<00:00, 30.84s/it]

0.6764207854866982


In [ ]:
with torch.inference_mode():
    n_batches = 0
    for betas, embeds in test_data_loader:
        test_iter_index += 1
        pred_embeds = model_embed(betas)

        loss_embed = mse_loss(pred_embeds, embeds)

        cos_embed = cos_loss(pred_embeds, embeds)

        writer.add_scalars('test/losses', {
            'embed': loss_embed.item(),
        }, test_iter_index)

        writer.add_scalars('test/cos_sim', {
            'cos_embed': cos_embed.item(),
        }, test_iter_index)

        n_batches += 1
        test_loss += loss_embed.item()